In [6]:
%cd ..

c:\Users\HP\OneDrive - University of Moratuwa\Desktop\E-Vision-Projects\Shelf_Product_Count_Generation


In [7]:
from transformers import CLIPProcessor, CLIPModel
import torch
import faiss
import numpy as np
from PIL import Image

In [8]:
from transformers import CLIPProcessor, CLIPModel
import torch
import faiss
import numpy as np
from PIL import Image

class ProductEmbeddingMatcher:
    def __init__(self, model_name="openai/clip-vit-base-patch32"):
        self.processor = CLIPProcessor.from_pretrained(model_name)
        self.model = CLIPModel.from_pretrained(model_name)
        self.model.eval()
        
    def get_embedding(self, image):
        """Extract embedding from product image"""
        inputs = self.processor(images=image, return_tensors="pt")
        with torch.no_grad():
            embedding = self.model.get_image_features(**inputs)
        return embedding.numpy().astype('float32')
    
    def build_faiss_index(self, reference_embeddings):
        """Build FAISS index for fast similarity search"""
        dimension = reference_embeddings.shape[1]
        index = faiss.IndexFlatL2(dimension)  # L2 distance
        index.add(reference_embeddings)
        return index
    
    def find_matches(self, query_embedding, index, top_k=5):
        """Find top-k similar products"""
        distances, indices = index.search(query_embedding, top_k)
        return distances, indices

In [48]:
from transformers import CLIPProcessor, CLIPModel
import torch
import faiss
import numpy as np
from PIL import Image

# Initialize CLIP
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
model.eval()

def get_embedding(image):
    """Extract normalized embedding from image"""
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        embedding = model.get_image_features(**inputs)
        # CLIP already normalizes, but ensure it
        embedding = embedding / embedding.norm(dim=-1, keepdim=True)
    return embedding.numpy().astype('float32')

# Build FAISS index with Inner Product (cosine similarity)
def build_cosine_index(embeddings):
    """Build FAISS index for cosine similarity search"""
    dimension = embeddings.shape[1]
    # IndexFlatIP = Inner Product (dot product) = cosine for normalized vectors
    index = faiss.IndexFlatIP(dimension)
    index.add(embeddings)
    return index

# Search with cosine similarity
def find_similar_products(query_embedding, index, product_ids, top_k=5):
    """Find top-k similar products using cosine similarity"""
    # Search returns dot product scores (higher = more similar)
    scores, indices = index.search(query_embedding.reshape(1, -1), top_k)
    
    results = []
    for i, (score, idx) in enumerate(zip(scores[0], indices[0])):
        # Score is already cosine similarity (0-1 range for normalized embeddings)
        results.append({
            'product_id': product_ids[idx],
            'similarity_score': float(score),  # This is cosine similarity
            'rank': i + 1
        })
    
    return results

In [49]:
import os
from pathlib import Path
from PIL import Image
import numpy as np

def load_reference_images_and_ids(reference_dir='data/reference_images'):
    """
    Load all reference images and create product_id mapping.
    
    Returns:
        embeddings: numpy array of all image embeddings
        product_ids: list of product_ids corresponding to each embedding
        image_paths: list of image paths for debugging
    """
    reference_path = Path(reference_dir)
    embeddings_list = []
    product_ids_list = []
    image_paths_list = []
    
    # Get all product folders (1000, 1001, 1002, etc.)
    product_folders = sorted([d for d in reference_path.iterdir() if d.is_dir()])
    
    print(f"Found {len(product_folders)} product folders")
    
    for product_folder in product_folders:
        product_id = product_folder.name  # e.g., "1000"
        
        # Get all images in this product folder
        image_files = sorted(product_folder.glob('*.jpg')) + sorted(product_folder.glob('*.jpeg')) + sorted(product_folder.glob('*.png'))
        
        print(f"  Product {product_id}: {len(image_files)} images")
        
        for image_path in image_files:
            try:
                # Load image
                image = Image.open(image_path).convert('RGB')
                
                # Get embedding
                embedding = get_embedding(image)
                
                embeddings_list.append(embedding)
                product_ids_list.append(product_id)
                image_paths_list.append(str(image_path))
                
            except Exception as e:
                print(f"    Error loading {image_path}: {e}")
                continue
    
    # Convert to numpy array
    embeddings = np.vstack(embeddings_list)
    
    print(f"\nTotal embeddings: {embeddings.shape[0]}")
    print(f"Embedding dimension: {embeddings.shape[1]}")
    
    return embeddings, product_ids_list, image_paths_list

# Load all reference images
reference_embeddings, product_ids, image_paths = load_reference_images_and_ids()

# Build FAISS index
index = build_cosine_index(reference_embeddings)

print(f"\nFAISS index built with {index.ntotal} vectors")

Found 49 product folders
  Product 1000: 3 images


  Product 1001: 4 images
  Product 1002: 4 images
  Product 1003: 3 images
  Product 1004: 5 images
  Product 1005: 2 images
  Product 1006: 5 images
  Product 1007: 3 images
  Product 1008: 2 images
  Product 1009: 3 images
  Product 1010: 2 images
  Product 1011: 2 images
  Product 1012: 3 images
  Product 1013: 2 images
  Product 1014: 2 images
  Product 1015: 2 images
  Product 1016: 2 images
  Product 1017: 2 images
  Product 1018: 2 images
  Product 1019: 2 images
  Product 1020: 2 images
  Product 15785: 3 images
  Product 15827: 3 images
  Product 15829: 6 images
  Product 15832: 5 images
  Product 58: 8 images
  Product 59: 4 images
  Product 60: 3 images
  Product 667: 5 images
  Product 668: 3 images
  Product 669: 4 images
  Product 670: 4 images
  Product 671: 2 images
  Product 672: 7 images
  Product 673: 3 images
  Product 674: 2 images
  Product 675: 3 images
  Product 677: 4 images
  Product 74: 4 images
  Product 75: 2 images
  Product 76: 4 images
  Product 77: 7 im

In [53]:
# Usage
query_embedding = get_embedding('data/test_images/cropped_image_3.jpg')
len(query_embedding[0])

512

In [54]:
results = find_similar_products(query_embedding, index, product_ids, top_k=5)

# Filter by threshold (e.g., 0.75 = 75% similarity)
matches = [r for r in results if r['similarity_score'] > 0.75]

In [55]:
matches

[{'product_id': '1002', 'similarity_score': 0.8852260708808899, 'rank': 1},
 {'product_id': '1003', 'similarity_score': 0.8810661435127258, 'rank': 2},
 {'product_id': '1003', 'similarity_score': 0.8785508871078491, 'rank': 3},
 {'product_id': '1005', 'similarity_score': 0.867941677570343, 'rank': 4},
 {'product_id': '1002', 'similarity_score': 0.8673974275588989, 'rank': 5}]